# Using Wallaroo to Discover Critical Computer Vision Insights

This tutorial demonstrates how to use the Wallaroo to detect objects in images through the following models:

* **rnn mobilenet**: A single stage object detector that performs fast inferences.  Mobilenet is typically good at identifying objects at a distance.
* **resnet50**:  A dual stage object detector with slower inferencing but but is able to detect objects that are closer to each other.

This tutorial series will demonstrate the following:

* Train a rnn mobilenet model and prepare it for use in Wallaroo.
* Train a resnet50 model and prepare it for use in Wallaroo.
* How to deploy a Wallaroo pipeline with trained rnn mobilenet model and perform sample inferences to detect objects in pictures, then display those objects.
* How to deploy a Wallaroo pipeline with a trained resnet50 model and perform sample inferences to detect objects in pictures, then display those objects.
* Use the Wallaroo feature shadow deploy to have both models perform inferences, then select the inference result with the higher confidence and show the objects detected.

## Prerequisites

Before beginning this tutorial, the following must be completed:

1. Install Wallaroo and have an account available for use.
1. This repository may use large file sizes for the models.  Verify that [Git Large File Storage (LFS)](https://git-lfs.com) is installed.